# Bay Area Highway Express Bus Corridor Trip Mapping

Summarize express bus trip by highway and interstate corridors leveraging pre-pandemic transit schedules.


## Methodology

**[Better Bus Buffers - Count Trips on Lines Documentation](https://github.com/Esri/public-transit-tools/blob/master/better-bus-buffers/UsersGuide.md#CountTripsOnLines)**

[Preprocessing of GTFS](https://github.com/Esri/public-transit-tools/blob/master/better-bus-buffers/UsersGuide.md#running-preprocess-gtfs) and [blank stop times interpolation](https://github.com/Esri/public-transit-tools/blob/master/interpolate-blank-stop-times/UsersGuide.md) was run for a previous analysis. the SQLLite databases were coppied into this project's directory and re-used. 

1. Preprocess transit lines using January 2020 GTFS data
2. Count trips on lines without combining routes for am peak period and for all-day period
3. Summarize by route and number of trips to get max trips per route during am period and all-day period
4. Generate pretty transit routes from routes.txt data 
5. Delete identical routes by route_id and shape
6. For routes with duplicate route_id, keep longest route line
    - Project pretty route lines to WGS84 UTM Zone 10 North
    - Summarize routes by route_id and max length
    - Join summary table to routes table 
    - Select routes w/ count > 1 and length <> max length and delete
    - Unjoin summary table
7. Join pretty routes and average trips per route table 
8. Create highway and interstate corridors
    - Project highway and interstate data to WGS84 UTM Zone 10 North
    - Buffer highway and interstate data by 50 meters and dissolve
    - Select junctions from highway and interstate data and create new feature class
    - Buffer junctions by 100 meters
    - Using editing tools, select all buffered junctions as input and highway and interstate features as target. Split and save edits. (This is a manual process- no arcpy geoprocessing tools available). 
9. Summarize transit trips within each highway and interstate corridor (**See notes below**)
    - Spatially-join express route trips to corridors
    - Summarize am peak period and all day trip counts by corridor
    - Join route trip summary table to highway corridor buffers
10. Create centerline from polygon highway corridor summary areas


In [1]:
import arcpy
import pandas as pd
import numpy as np
import os
from arcgis.geometry import Geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

#Define the workspace
arcpy.env.workspace = r'Bay_Area_Forward_Transit_Database.gdb'

#Allow features and files to be overwritten 
arcpy.env.overwriteOutput = True

## Import custom toolboxes

In [2]:
#Better Bus Buffers
bbb = r'BetterBusBuffers_0.14.0.0\BetterBusBuffers.pyt'
arcpy.ImportToolbox(bbb)

#Interpolate Blank Stops
ibs = r'InterpolateBlankStopTimes_0.1.2.1\InterpolateBlankStopTimes.tbx'
arcpy.ImportToolbox(ibs)

<module 'transit'>

## Step 1- Preprocess Lines

GTFS SQL Database contains data for January 2020 w/ blank stops interpolated during another process

In [3]:
route_temp_fc = 'bay_area_route_line_template_01_2020'
sql_database = r'bay_area_gtfs.sql'
arcpy.BetterBusBuffers.BBBLines_PreprocessLines(output_template_feature_class=route_temp_fc,
                                                sql_database=sql_database,
                                                combine_routes=False)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\bay_area_route_line_template_01_2020'>

## Step 2- Count Trips on Lines

In [4]:
#Count trips during AM Peak Period (5:00 - 10:00)
transit_rt_trip_ct_am = 'trip_count_am_period_01_2020'
day = '20200106'
time_window_start = '05:00'
time_window_end = '10:00'
arcpy.BetterBusBuffers.BBBLines_CountTripsOnLines(input_template_feature_class=route_temp_fc,
                                                  sql_database=sql_database, 
                                                  output_feature_class=transit_rt_trip_ct_am,
                                                  day=day,
                                                  time_window_start=time_window_start, 
                                                  time_window_end=time_window_end)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\trip_count_am_period_01_2020'>

In [5]:
#Rename numtrips field
arcpy.AlterField_management(in_table=transit_rt_trip_ct_am,
                            field='NumTrips',
                            new_field_name='am_num_trips',
                            new_field_alias='am_num_trips')

<Result 'trip_count_am_period_01_2020'>

In [6]:
#Count trips all day
transit_rt_trip_ct_allday = 'trip_count_all_day_01_2020'
day = '20200106'
time_window_start = '00:00'
time_window_end = '23:59'
arcpy.BetterBusBuffers.BBBLines_CountTripsOnLines(input_template_feature_class=route_temp_fc,
                                                  sql_database=sql_database, 
                                                  output_feature_class=transit_rt_trip_ct_allday,
                                                  day=day,
                                                  time_window_start=time_window_start, 
                                                  time_window_end=time_window_end)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\trip_count_all_day_01_2020'>

In [7]:
#Rename numtrips field
arcpy.AlterField_management(in_table=transit_rt_trip_ct_allday,
                            field='NumTrips',
                            new_field_name='allday_num_trips',
                            new_field_alias='allday_num_trips')

<Result 'trip_count_all_day_01_2020'>

In [8]:
#Copy am trips fc to new fc 
transit_rt_trip_ct = "trip_count_am_all_day_01_2020"
arcpy.FeatureClassToFeatureClass_conversion(in_features=transit_rt_trip_ct_am,
                                            out_name=transit_rt_trip_ct)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\trip_count_am_all_day_01_2020'>

In [9]:
#Delete un-needed fields
del_fields = ["NumTripsPerHr","MaxWaitTime","AvgHeadway"]
arcpy.DeleteField_management(in_table=transit_rt_trip_ct,
                             drop_field=del_fields)

<Result 'trip_count_am_all_day_01_2020'>

In [10]:
#Join am_period and all day features
arcpy.JoinField_management(in_data=transit_rt_trip_ct,
                           in_field='pair_id',
                           join_table=transit_rt_trip_ct_allday,
                           join_field='pair_id',fields='allday_num_trips')

<Result 'trip_count_am_all_day_01_2020'>

In [11]:
#Add field which extracts short route id from route id column
field_name = 'route_id_short' 
field_type = 'TEXT'
arcpy.management.AddField(in_table=transit_rt_trip_ct,
                          field_name=field_name,
                          field_type=field_type)

<Result 'trip_count_am_all_day_01_2020'>

In [12]:
field = 'route_id_short'
expression = "extract_routeid('202001:(.*$)',!route_id!)"
code_block = """def extract_routeid(regex,string):
    import re
    y = re.search(regex,string)
    if y != None:
        return y.group(1)
    else:
        return None"""
arcpy.management.CalculateField(in_table=transit_rt_trip_ct,
                                field=field,
                                expression=expression,
                                expression_type='PYTHON3',
                                code_block=code_block)

<Result 'trip_count_am_all_day_01_2020'>

## Step 3- Summarize by route and number of trips to get max trips per route during am period

In [13]:
fields = ["route_id_short"]
statistics_fields = [["am_num_trips","MAX"],["allday_num_trips","MAX"]]
max_route_ct_by_route = 'max_route_ct_by_route_01_2020'

arcpy.Statistics_analysis(in_table=transit_rt_trip_ct,
                          out_table=max_route_ct_by_route,
                          statistics_fields=statistics_fields,
                          case_field=fields)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\max_route_ct_by_route_01_2020'>

## Step 4- Generate pretty routes from routes.txt

In [14]:
!dir "Historic GTFS/2020-01/"

 Volume in drive Z is Shared Folders
 Volume Serial Number is 0000-0000

 Directory of Z:\Documents\ArcGIS\Projects\Bay_Area_Forward_Transit_Mapping\Historic GTFS\2020-01

11/30/1979  01:00 AM            76,670 fare_attributes.txt
11/30/1979  01:00 AM             3,751 agency.txt
11/30/1979  01:00 AM            86,435 fare_rules.txt
11/30/1979  01:00 AM       174,808,847 calendar_dates.txt
01/18/2021  12:07 PM       570,857,707 stop_times.txt
11/30/1979  01:00 AM       325,134,902 shapes.txt
11/30/1979  01:00 AM        37,437,443 trips.txt
11/30/1979  01:00 AM            54,972 feed_info.txt
11/30/1979  01:00 AM         1,777,863 stops.txt
01/18/2021  12:07 PM               644 stop_times.txt.xml
11/30/1979  01:00 AM            71,290 routes.txt
              11 File(s)  1,110,310,524 bytes
               0 Dir(s)  1,795,857,833,984 bytes free


In [15]:
transit_rts = os.path.join('Z:',
                           '\Documents',
                           'ArcGIS',
                           'Projects',
                           'Bay_Area_Forward_Transit_Mapping',
                           'Historic GTFS',
                           '2020-01',
                           'shapes.txt')
transit_rts_fc = "pretty_transit_route_line_01_2020"
arcpy.GTFSShapesToFeatures_conversion(in_gtfs_shapes_file=transit_rts,
                                      out_feature_class=transit_rts_fc)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\pretty_transit_route_line_01_2020'>

## Step 5- Delete identical routes

In [16]:
arcpy.management.DeleteIdentical(in_dataset=transit_rts_fc,
                                 fields=["route_id","Shape"])

<Result 'pretty_transit_route_line_01_2020'>

## Step 6- For routes with duplicate route_id, keep longest route line

In [17]:
transit_routs_proj_fc = "pretty_transit_route_line_proj_01_2020"
sr = arcpy.SpatialReference(32610)
arcpy.Project_management(in_dataset=transit_rts_fc,
                         out_dataset=transit_routs_proj_fc,
                         out_coor_system=sr)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\pretty_transit_route_line_proj_01_2020'>

In [18]:
#summarize by route and length max
route_count_max_length = "route_count_max_length_summary"
statistics_fields = [["route_id","COUNT"],
                    ["Shape_Length","MAX"]]
arcpy.Statistics_analysis(in_table=transit_routs_proj_fc,
                          out_table=route_count_max_length,
                          statistics_fields=statistics_fields,
                          case_field=["route_id"])

<Result 'Bay_Area_Forward_Transit_Database.gdb\\route_count_max_length_summary'>

In [19]:
#join summary table to route table
arcpy.management.AddJoin(in_layer_or_view=transit_routs_proj_fc,
                         in_field="route_id",
                         join_table=route_count_max_length,
                         join_field="route_id")

<Result 'pretty_transit_route_line_proj_01_2020'>

In [20]:
#Select routes with counts over 1 and where length <> max length
where_clause = "route_count_max_length_summary.COUNT_route_id > 1\
and Shape_Length <> route_count_max_length_summary.MAX_Shape_Length"
arcpy.SelectLayerByAttribute_management(in_layer_or_view=transit_routs_proj_fc,
                                        where_clause=where_clause)

id,value
0,a Layer object
1,2297


In [21]:
#Delete shortest routes
arcpy.management.DeleteRows(in_rows=transit_routs_proj_fc)

<Result 'pretty_transit_route_line_proj_01_2020'>

In [22]:
#Remove join
arcpy.management.RemoveJoin(in_layer_or_view=transit_routs_proj_fc)

<Result 'pretty_transit_route_line_proj_01_2020'>

## Step 7- Join pretty transit routes and max route count summary table

In [23]:
#add trip count fields
arcpy.JoinField_management(in_data=transit_routs_proj_fc,
                           in_field="route_id",
                           join_table=max_route_ct_by_route, 
                           join_field="route_id_short",
                           fields=["MAX_am_num_trips","MAX_allday_num_trips"])

<Result 'pretty_transit_route_line_proj_01_2020'>

In [24]:
#add express route field
express_route_lookup = "express_route_lookup"
arcpy.JoinField_management(in_data=transit_routs_proj_fc,
                           in_field="route_id",
                           join_table=express_route_lookup,
                           join_field="route_id",
                           fields="express_route")

<Result 'pretty_transit_route_line_proj_01_2020'>

In [25]:
#export routes and route counts to new fc
express_routes = "express_route_trips"
where = "express_route = 1"
arcpy.Select_analysis(in_features=transit_routs_proj_fc,
                      out_feature_class=express_routes,
                      where_clause=where)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\express_route_trips'>

## Step 8- Create highway and interstate corridors

In [37]:
# #Project to WGS84 UTM Zone 10 North 
# road_highway_interstate = "road_highway_interstate"
# sr = arcpy.SpatialReference(32610)
# roads_proj = "road_highway_interstate_proj"
# with arcpy.EnvManager(outputCoordinateSystem=sr):
#     arcpy.management.CopyFeatures(in_features=road_highway_interstate,
#                                   out_feature_class=roads_proj)

In [31]:
#Buffer highways and interstates by 50 meters and dissolve
roads_proj = "road_highway_interstate_proj"
roads_buff = "road_highway_interstate_corridor_50m_buffer"
arcpy.Buffer_analysis(in_features=roads_proj,
                      out_feature_class=roads_buff,
                      buffer_distance_or_field="50 Meters",
                      line_side="FULL",
                      line_end_type="ROUND",
                      dissolve_option="ALL")

<Result 'Bay_Area_Forward_Transit_Database.gdb\\road_highway_interstate_corridor_50m_buffer'>

In [32]:
#Select all junctions from interstates and highways
where_clause = "ramp IN (1, 2)"
junctions = "highway_interstate_junctions"
arcpy.Select_analysis(in_features=roads_proj,
                      out_feature_class=junctions,
                      where_clause=where_clause)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\highway_interstate_junctions'>

In [33]:
junctions_buff = "highway_interstate_junctions_100m_buffer"
arcpy.Buffer_analysis(in_features=junctions,
                      out_feature_class=junctions_buff,
                      buffer_distance_or_field="100 Meters",
                      line_side="FULL",
                      line_end_type="FLAT",
                      dissolve_option="ALL")

<Result 'Bay_Area_Forward_Transit_Database.gdb\\highway_interstate_junctions_100m_buffer'>

## Step 9- Summarize within highway corridor areas

In [56]:
# express_route_corridor_summary = "express_route_corridor_summary"
# sum_fields = [["MAX_am_num_trips","Sum"],["MAX_allday_num_trips","Sum"]]
# arcpy.SummarizeWithin_analysis(in_polygons="road_highway_interstate_corridor_50m_buffer",
#                                in_sum_features=express_routes,
#                                out_feature_class=express_route_corridor_summary,
#                                keep_all_polygons="ONLY_INTERSECTING",
#                                sum_fields=sum_fields,
#                                sum_shape="NO_SHAPE_SUM")

In [34]:
#Spatial join express route trips to corridors
express_route_corridor = "express_route_corridor"
express_routes = "express_route_trips"
roads_buff = "road_highway_interstate_corridor_50m_buffer"
arcpy.SpatialJoin_analysis(target_features=roads_buff,
                           join_features=express_routes,
                           out_feature_class=express_route_corridor,
                           join_operation="JOIN_ONE_TO_MANY",
                           join_type="KEEP_COMMON",
                           match_option="INTERSECT")

<Result 'Bay_Area_Forward_Transit_Database.gdb\\express_route_corridor'>

In [35]:
#Summarize corridor trips by corridor
express_route_corridor_summary = "express_route_corridor_summary"
fields = ["TARGET_FID"]
sum_fields = [["MAX_am_num_trips","SUM"],["MAX_allday_num_trips","SUM"]]
arcpy.SummarizeAttributes_gapro(input_layer=express_route_corridor,
                                out_table=express_route_corridor_summary,
                                fields=fields,
                                summary_fields=sum_fields)

<Result 'Z:\\Documents\\ArcGIS\\Projects\\Bay_Area_Forward_Transit_Mapping\\Bay_Area_Forward_Transit_Database.gdb\\express_route_corridor_summary'>

In [37]:
#rename columns
namesDict = {"Sum_MAX_am_num_trips":"am_num_trips",
            "Sum_MAX_allday_num_trips":"allday_num_trips"}

[arcpy.AlterField_management(express_route_corridor_summary, f, namesDict[f], namesDict[f]) for f in namesDict]

[<Result 'express_route_corridor_summary'>, <Result 'express_route_corridor_summary'>]

In [38]:
#Join trip summary table to features
roads_buff = "road_highway_interstate_corridor_50m_buffer"
arcpy.AddJoin_management(in_layer_or_view=roads_buff,
                         in_field="OBJECTID",
                         join_table=express_route_corridor_summary,
                         join_field="TARGET_FID",
                         join_type="KEEP_COMMON")

<Result 'road_highway_interstate_corridor_50m_buffer'>

In [39]:
#copy joined features to new fc
express_route_corridor_fc = "express_route_corridor_summary_fc"
arcpy.FeatureClassToFeatureClass_conversion(in_features=roads_buff,
                                            out_name=express_route_corridor_fc)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\express_route_corridor_summary_fc'>

In [40]:
arcpy.RemoveJoin_management(in_layer_or_view=roads_buff)

<Result 'road_highway_interstate_corridor_50m_buffer'>

In [41]:
arcpy.DeleteField_management(in_table=express_route_corridor_fc,
                             drop_field=["OBJECTID","TARGET_FID"])

<Result 'express_route_corridor_summary_fc'>

## Step 10- Create centerline from polygon highway corridor summary areas

In [42]:
express_route_corridor_line = "express_route_corridor_line_fc"
arcpy.PolygonToCenterline_topographic(in_features=express_route_corridor_fc,
                                      out_feature_class=express_route_corridor_line)

<Result 'Bay_Area_Forward_Transit_Database.gdb\\express_route_corridor_line_fc'>

In [44]:
arcpy.JoinField_management(in_data=express_route_corridor_line,
                           in_field="FID",
                           join_table=express_route_corridor_fc,
                           join_field="OBJECTID_1",
                           fields=["COUNT","am_num_trips","allday_num_trips"])

<Result 'express_route_corridor_line_fc'>